# US Stock News Summarizer V2 - Deep Analysis

This notebook provides in-depth analysis of stock news from multiple business perspectives:
- Product Strategy
- Financial Impact
- Growth & Competition
- Valuation & Investment

In [ ]:
# Install required packages (run once)
# !pip install openai feedparser requests python-dotenv yfinance

In [ ]:
import feedparser
import requests
from openai import OpenAI
from datetime import datetime
import os
from dotenv import load_dotenv
import yfinance as yf
import json

# Load environment variables from .env file
load_dotenv()

In [ ]:
# Initialize OpenAI client
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file. Please add it to your .env file.")

client = OpenAI(api_key=api_key)
print("OpenAI client initialized successfully!")

## 1. Data Collection Functions

In [ ]:
def fetch_stock_news(ticker, num_articles=10):
    """
    Fetch latest news for a specific stock ticker from Google News RSS
    
    Args:
        ticker: Stock ticker symbol (e.g., 'TSLA', 'TSM')
        num_articles: Number of articles to fetch
    
    Returns:
        List of article dictionaries
    """
    url = f'https://news.google.com/rss/search?q={ticker}+stock&hl=en-US&gl=US&ceid=US:en'
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:num_articles]:
        article = {
            'ticker': ticker,
            'title': entry.title,
            'link': entry.link,
            'published': entry.published if 'published' in entry else 'N/A',
            'summary': entry.summary if 'summary' in entry else entry.title
        }
        articles.append(article)
    
    return articles

In [ ]:
def fetch_company_financials(ticker):
    """
    Fetch key financial metrics for a stock using yfinance
    
    Args:
        ticker: Stock ticker symbol
    
    Returns:
        Dictionary of financial metrics
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        financials = {
            'company_name': info.get('longName', ticker),
            'sector': info.get('sector', 'N/A'),
            'industry': info.get('industry', 'N/A'),
            'market_cap': info.get('marketCap', 'N/A'),
            'pe_ratio': info.get('trailingPE', 'N/A'),
            'forward_pe': info.get('forwardPE', 'N/A'),
            'revenue': info.get('totalRevenue', 'N/A'),
            'revenue_growth': info.get('revenueGrowth', 'N/A'),
            'profit_margin': info.get('profitMargins', 'N/A'),
            'operating_margin': info.get('operatingMargins', 'N/A'),
            'roe': info.get('returnOnEquity', 'N/A'),
            'debt_to_equity': info.get('debtToEquity', 'N/A'),
            'current_price': info.get('currentPrice', 'N/A'),
            'target_price': info.get('targetMeanPrice', 'N/A'),
            'analyst_recommendation': info.get('recommendationKey', 'N/A'),
            'earnings_growth': info.get('earningsGrowth', 'N/A'),
            'book_value': info.get('bookValue', 'N/A'),
            'price_to_book': info.get('priceToBook', 'N/A')
        }
        
        return financials
    except Exception as e:
        print(f"Error fetching financials for {ticker}: {e}")
        return {}

In [ ]:
def format_financials_for_analysis(financials):
    """
    Format financial data into readable text for AI analysis
    """
    if not financials:
        return "Financial data not available"
    
    def format_number(value):
        if value == 'N/A' or value is None:
            return 'N/A'
        if isinstance(value, (int, float)):
            if value > 1_000_000_000:
                return f"${value/1_000_000_000:.2f}B"
            elif value > 1_000_000:
                return f"${value/1_000_000:.2f}M"
            elif value > 1000:
                return f"${value/1000:.2f}K"
            elif abs(value) < 100:
                return f"{value:.2f}"
            else:
                return f"${value:.2f}"
        return str(value)
    
    def format_percentage(value):
        if value == 'N/A' or value is None:
            return 'N/A'
        if isinstance(value, (int, float)):
            return f"{value*100:.2f}%"
        return str(value)
    
    text = f"""
Company: {financials.get('company_name', 'N/A')}
Sector: {financials.get('sector', 'N/A')}
Industry: {financials.get('industry', 'N/A')}

Valuation Metrics:
- Market Cap: {format_number(financials.get('market_cap'))}
- Current Price: {format_number(financials.get('current_price'))}
- Target Price: {format_number(financials.get('target_price'))}
- P/E Ratio: {financials.get('pe_ratio', 'N/A')}
- Forward P/E: {financials.get('forward_pe', 'N/A')}
- Price to Book: {financials.get('price_to_book', 'N/A')}
- Book Value: {format_number(financials.get('book_value'))}

Financial Performance:
- Total Revenue: {format_number(financials.get('revenue'))}
- Revenue Growth: {format_percentage(financials.get('revenue_growth'))}
- Earnings Growth: {format_percentage(financials.get('earnings_growth'))}
- Profit Margin: {format_percentage(financials.get('profit_margin'))}
- Operating Margin: {format_percentage(financials.get('operating_margin'))}
- Return on Equity: {format_percentage(financials.get('roe'))}
- Debt to Equity: {financials.get('debt_to_equity', 'N/A')}

Analyst View:
- Recommendation: {financials.get('analyst_recommendation', 'N/A')}
"""
    return text

## 2. Deep Analysis Functions

Each function analyzes news from a specific business perspective

In [ ]:
def analyze_product_perspective(ticker, articles, financials):
    """
    Analyze news from product strategy perspective
    """
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    financial_context = format_financials_for_analysis(financials)
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a senior product strategist analyzing how news impacts a company's product portfolio and strategy.
                
Focus on:
1. Product Innovation & Development: New products, features, R&D initiatives
2. Product-Market Fit: How products address market needs and customer pain points
3. Technology Leadership: Technical capabilities and competitive advantages
4. Product Roadmap Impact: How news affects future product plans
5. Customer Adoption: Signals about product acceptance and usage

Provide actionable insights with specific examples from the news."""
            },
            {
                "role": "user",
                "content": f"""Analyze {ticker} from a PRODUCT PERSPECTIVE based on:

RECENT NEWS:
{news_text}

FINANCIAL CONTEXT:
{financial_context}

Provide a comprehensive product strategy analysis."""
            }
        ],
        max_tokens=1000,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
def analyze_financial_perspective(ticker, articles, financials):
    """
    Analyze news from financial performance perspective
    """
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    financial_context = format_financials_for_analysis(financials)
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a financial analyst examining how news impacts a company's financial performance and health.
                
Focus on:
1. Revenue Impact: How news affects top-line growth and revenue streams
2. Profitability: Implications for margins, costs, and bottom-line performance
3. Cash Flow: Impact on operating cash flow and capital allocation
4. Financial Risk: Changes to debt levels, liquidity, or financial stability
5. Capital Efficiency: ROE, ROIC, and resource utilization

Use specific numbers from the financial data to support your analysis."""
            },
            {
                "role": "user",
                "content": f"""Analyze {ticker} from a FINANCIAL PERSPECTIVE based on:

RECENT NEWS:
{news_text}

FINANCIAL METRICS:
{financial_context}

Provide a comprehensive financial impact analysis."""
            }
        ],
        max_tokens=1000,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
def analyze_growth_competition_perspective(ticker, articles, financials):
    """
    Analyze news from growth and competitive positioning perspective
    """
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    financial_context = format_financials_for_analysis(financials)
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a strategic analyst focusing on growth trajectory and competitive dynamics.
                
Focus on:
1. Market Share: Changes in competitive position and market penetration
2. Growth Drivers: Key factors enabling or constraining growth
3. Competitive Advantages: Moats, barriers to entry, and sustainable advantages
4. Competitive Threats: Emerging competitors, disruption risks, market shifts
5. Market Expansion: New markets, geographies, or customer segments
6. Industry Trends: How the company is positioned relative to sector trends

Identify both opportunities and threats with specific examples."""
            },
            {
                "role": "user",
                "content": f"""Analyze {ticker} from a GROWTH & COMPETITION PERSPECTIVE based on:

RECENT NEWS:
{news_text}

FINANCIAL CONTEXT:
{financial_context}

Provide a comprehensive growth and competitive analysis."""
            }
        ],
        max_tokens=1000,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
def analyze_valuation_perspective(ticker, articles, financials):
    """
    Analyze news from valuation and investment perspective
    """
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    financial_context = format_financials_for_analysis(financials)
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are an investment analyst evaluating valuation and investment implications.
                
Focus on:
1. Current Valuation: Assessment of P/E, P/B, and other multiples relative to fundamentals
2. Fair Value Impact: How news affects intrinsic value estimates
3. Risk-Reward: Changes to investment risk profile and potential returns
4. Sentiment & Momentum: Market perception and technical factors
5. Peer Comparison: Valuation relative to competitors and sector
6. Investment Thesis: How news supports or challenges the bull/bear case

Reference specific valuation metrics and provide balanced perspective."""
            },
            {
                "role": "user",
                "content": f"""Analyze {ticker} from a VALUATION & INVESTMENT PERSPECTIVE based on:

RECENT NEWS:
{news_text}

VALUATION METRICS:
{financial_context}

Provide a comprehensive valuation and investment analysis."""
            }
        ],
        max_tokens=1000,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
def generate_executive_summary(ticker, articles, financials, analyses):
    """
    Generate an executive summary synthesizing all perspectives
    """
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}"
        for i, article in enumerate(articles[:5])  # Top 5 headlines
    ])
    
    all_analyses = f"""
PRODUCT ANALYSIS:
{analyses['product']}

FINANCIAL ANALYSIS:
{analyses['financial']}

GROWTH & COMPETITION ANALYSIS:
{analyses['growth_competition']}

VALUATION ANALYSIS:
{analyses['valuation']}
"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a senior executive advisor synthesizing complex analysis into clear, actionable insights.
                
Create a concise executive summary that:
1. Highlights the 3-5 most critical takeaways across all perspectives
2. Identifies key opportunities and risks
3. Provides a balanced overall assessment
4. Suggests areas requiring attention or further monitoring

Keep it concise and decision-focused."""
            },
            {
                "role": "user",
                "content": f"""Create an executive summary for {ticker} based on:

TOP HEADLINES:
{news_text}

DETAILED ANALYSES:
{all_analyses}

Provide a clear, actionable executive summary."""
            }
        ],
        max_tokens=800,
        temperature=0.7
    )
    
    return response.choices[0].message.content

## 3. Main Analysis Pipeline

In [ ]:
def analyze_stock_deep(ticker, num_articles=15):
    """
    Complete deep analysis pipeline for a single stock
    
    Args:
        ticker: Stock ticker symbol
        num_articles: Number of news articles to analyze
    
    Returns:
        Dictionary containing all analyses
    """
    print(f"\n{'='*80}")
    print(f"DEEP ANALYSIS: {ticker}")
    print(f"{'='*80}\n")
    
    # 1. Fetch data
    print("📰 Fetching news articles...")
    articles = fetch_stock_news(ticker, num_articles)
    print(f"   Found {len(articles)} articles\n")
    
    print("💰 Fetching financial data...")
    financials = fetch_company_financials(ticker)
    print(f"   Retrieved financial metrics for {financials.get('company_name', ticker)}\n")
    
    # 2. Run analyses
    analyses = {}
    
    print("🔍 Analyzing from PRODUCT perspective...")
    analyses['product'] = analyze_product_perspective(ticker, articles, financials)
    
    print("💵 Analyzing from FINANCIAL perspective...")
    analyses['financial'] = analyze_financial_perspective(ticker, articles, financials)
    
    print("📈 Analyzing from GROWTH & COMPETITION perspective...")
    analyses['growth_competition'] = analyze_growth_competition_perspective(ticker, articles, financials)
    
    print("💎 Analyzing from VALUATION perspective...")
    analyses['valuation'] = analyze_valuation_perspective(ticker, articles, financials)
    
    print("📋 Generating executive summary...\n")
    analyses['executive_summary'] = generate_executive_summary(ticker, articles, financials, analyses)
    
    # 3. Display results
    print("\n" + "="*80)
    print(f"EXECUTIVE SUMMARY - {ticker}")
    print("="*80)
    print(analyses['executive_summary'])
    
    print("\n\n" + "="*80)
    print("PRODUCT PERSPECTIVE")
    print("="*80)
    print(analyses['product'])
    
    print("\n\n" + "="*80)
    print("FINANCIAL PERSPECTIVE")
    print("="*80)
    print(analyses['financial'])
    
    print("\n\n" + "="*80)
    print("GROWTH & COMPETITION PERSPECTIVE")
    print("="*80)
    print(analyses['growth_competition'])
    
    print("\n\n" + "="*80)
    print("VALUATION & INVESTMENT PERSPECTIVE")
    print("="*80)
    print(analyses['valuation'])
    
    print("\n" + "="*80 + "\n")
    
    return {
        'ticker': ticker,
        'articles': articles,
        'financials': financials,
        'analyses': analyses
    }

In [ ]:
def analyze_multiple_stocks_deep(tickers, num_articles=15):
    """
    Run deep analysis for multiple stocks
    
    Args:
        tickers: List of stock ticker symbols
        num_articles: Number of articles per stock
    
    Returns:
        Dictionary with results for each stock
    """
    results = {}
    
    for ticker in tickers:
        results[ticker] = analyze_stock_deep(ticker, num_articles)
    
    return results

## 4. Run Analysis

Execute deep analysis for your selected stocks

In [ ]:
# Analyze Tesla and Taiwan Semiconductor
results = analyze_multiple_stocks_deep(['TSLA', 'TSM'], num_articles=15)

In [ ]:
# Analyze drone/defense stocks
results_defense = analyze_multiple_stocks_deep(['ONDS', 'RCAT', 'AVAV'], num_articles=20)

## 5. Export Results (Optional)

Save analysis results to JSON for further processing

In [ ]:
def export_analysis(results, filename):
    """
    Export analysis results to JSON file
    """
    export_data = {}
    
    for ticker, data in results.items():
        export_data[ticker] = {
            'company_name': data['financials'].get('company_name', ticker),
            'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'num_articles_analyzed': len(data['articles']),
            'key_metrics': {
                'market_cap': data['financials'].get('market_cap'),
                'pe_ratio': data['financials'].get('pe_ratio'),
                'revenue_growth': data['financials'].get('revenue_growth')
            },
            'analyses': data['analyses']
        }
    
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"✅ Analysis exported to {filename}")

In [ ]:
# Export results
# export_analysis(results, 'stock_analysis_TSLA_TSM.json')
# export_analysis(results_defense, 'stock_analysis_defense.json')

## 6. Quick Analysis Function

For quick single-stock analysis

In [ ]:
# Quick analysis for any stock
# analyze_stock_deep('NVDA', num_articles=15)